In [1]:
"""
This code is for genberating a list of dates where a potential vorticity cutoff from 
the dataset published in Portmann et al. 2021 (https://doi.org/10.5194/wcd-2-507-2021)
occurs in the SouthEastern Australia study region (for comparison with Pook et al. 2006). 
"""
import numpy as np
import xarray as xr
from datetime import datetime
import matplotlib.dates as dt

In [2]:
# Going to need a num2date conversion function here
def date_from_pmtime(year, n):
    
    date_0 = datetime(year,1,1)
    num0 = dt.date2num(date_0)
    
    num = num0 + ((n) / 4)
    
    date_obj = dt.num2date(num)
    
    #repr(date_obj)
    
    return date_obj 

In [20]:
# ========================== Set Initial Parameters ================================
save_path = '/home/561/nxg561/00_Tracking_Scheme_Comparison/Input_Data/Portmann_Lows/Sup/'

# Set the lat and lon bounds here
lats = (-30, -45)
lons = (125, 147)

# Define a buffer zone within the study region for the pv cutoff to overlap
buf = 7 # Integer between 0 and 7

In [21]:
# Open the array of Portmann fields and extract the numpy array
portmann_dataset_filename = '/g/data/w40/nxg561/Portmann_Cutoffs/300_320/pv_cutoffs_1979-2017.nc'
xr_obj_pm = xr.open_dataset(portmann_dataset_filename, engine = 'netcdf4')

# Extract the cutoff indicator fields within the SEA study region here, applying the specified buffer zone
pm_array = xr_obj_pm['track_id'].sel(latitude = slice(lats[1] + buf,lats[0] - buf), longitude = slice(lons[0] + buf, lons[1] - buf))

# Sum the cutoff indicator values within the study area for each timestep. If the result is greater than 0
# for the spatial subset, it indicates a pv cutoff in that region on that date. 
sum_array = pm_array.sum(dim = ['latitude', 'longitude'])

In [22]:
# Get the numpy array out of the dataarray of cutoff indicators sum_array
cutoffs = sum_array.data

# Find the time steps with a cutoff indicator inside the field
time_inds = np.where(cutoffs > 0)

# get the number of rows for the new array
rows = np.shape(time_inds)[1]

In [23]:
# Compute the y,m,d of the dates with a cutoff in the region, write the info in
# a numpy array, and save as a .txt file. 

date_array = np.zeros((rows, 3))

for ii in range(rows):

    # The time axis indices will be based on the first year of the time scale. 
    ii_date = date_from_pmtime(1979, time_inds[0][ii])

    y = ii_date.year
    m = ii_date.month
    d = ii_date.day

    date_array[ii,0] = y
    date_array[ii,1] = m
    date_array[ii,2] = d

#output_filename = save_path + 'pv_cutoffs_sea_' + str(buf) + '_latitude.txt'
output_filename = save_path + 'pv_cutoffs_sea_' + str(buf) + '.txt'
np.savetxt(output_filename, date_array, delimiter = ',')

In [19]:
# this is where I write some code that counts the number of cells that are greater than 0
# for each date in the SEA region. If I run this for both datasets of a single isentrope for each 
# latitude and a range of isentropes across the whole domain, I can see if the single isentrope dataset simply 
# has most of the same systems, but just covering smaller areas. 
print(np.shape(date_array))
print(np.sum(sum_array.data))

(13138, 3)
295580.0


In [14]:
np.shape(pm_array)

(56980, 6, 13)

In [15]:
sum_array

<xarray.DataArray 'track_id' (time: 56980)>
array([ 0.,  0.,  0., ..., 20.,  1.,  3.], dtype=float32)
Dimensions without coordinates: time